In [2]:
!pip3 install easydict

  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6506 sha256=374dfd4a4d1834a7fcd3c453be0f63aadbe5e6602e839b4ca46a142334e426f9
  Stored in directory: /root/.cache/pip/wheels/8d/d6/16/3fd964549d5c27d89d6c5fdb4306283ca55be1799f3d48a67b
Successfully built easydict


In [95]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import easydict
import matplotlib.pyplot as plt
from PIL import Image

module_path = os.path.abspath('/ws/external')
if module_path not in sys.path:
    sys.path.append(module_path)
from datasets.builder import build_dataset
from datasets.harddataset import HardDataset

config = easydict.EasyDict({
    'batch_size': 1
})

###############
### Dataset ###
###############
args = easydict.EasyDict({
    'dataset': 'cifar10',
    'aug': 'augmix',
    'no_jsd': False,
    'all_ops': False,
    'mixture_width': 3,
    'mixture_depth': -1,
    'aug_severity': 3,
    'mixture_coefficient': 1.0,
})
hard_cifar10_dir = '/ws/data/log/cifar10/baselines/augmix_b256_jsdv3_lw12/false_images'
train_dataset, test_dataset, num_classes, base_c_path = build_dataset(args)
train_dataset = HardDataset(train_dataset, hard_cifar10_dir)

train_loader = torch.utils.data.DataLodar(train_dataset,
                                          batch_size)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [84]:
parent_dir = '/ws/data/log/cifar10/baselines/augmix_b256_jsdv3_lw12/false_images'
file_list = os.listdir(parent_dir)

files = dict()
for filename in file_list:
    fn = filename.split('_')
    
    index = int(fn[0][1:])
    corruption = '_'.join(fn[1:-3])
    severity = int(fn[-3][1:])
    target = int(fn[-2][1:])
    pred = int(fn[-1].split('.png')[0][1:])
    
    if not index in files:
        files[index] = []
        
    files[index].append(
        {'corruption':corruption, 
         'severity':severity, 
         'target':target, 
         'pred':pred,
         'filename':filename})
    
    # print(fn)

# print(files)


In [87]:
index = 96
index = 97
if index in files.keys():
    files_ = files[index]
    for file in files_:
        print(file['filename'])
        break
else:
    print('no')

f97_frost_s4_t0_p2.png


In [69]:
key, values = sorted_files[6]
key

6